In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import pandas as pd 
from scipy import optimize
from scipy import integrate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

sns.set(style="darkgrid")

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)




# Fitting the parameters of SIR model

In [2]:
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

In [3]:
def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), tnew, args=(beta, gamma))[:,1] # we only would like to get dI

In [7]:
def SIR_model_Large(ydatanew):
  
    N0 = 1000000
    global S0,I0,R0,tnew
    
    I0=ydatanew[0]
    
    S0=N0-I0
    R0=0
    SIR =(S0,I0,R0)
   
    popt = [0.4, 0.1]
    tnew=np.arange(len(ydatanew))
    fit_odeint(tnew, *popt)
    popt, pcov = optimize.curve_fit(fit_odeint, tnew, ydatanew)
    fitted=fit_odeint(tnew, *popt)
    return fitted


In [8]:
df_input_large=pd.read_csv('../data/processed/COVID_final_set.csv',sep=';')
df_input_large=df_input_large.sort_values('date',ascending=True)
#take cases where infected people are more than 30
df_input_large=df_input_large[df_input_large['confirmed'] > 30]
#print(df_input_large)

In [9]:
fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Fitting the parameters of SIR model

    '''),

    dcc.Markdown('''
    ## Select Country for visualization
    '''),


    dcc.Dropdown(
        id = 'country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value= 'Germany', # which are pre-selected
        multi=False
    ),

    
    dcc.Graph(figure=fig, id='SIR_fitting')
])

@app.callback(
    Output('SIR_fitting', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure_SIR(country_drop_down):


    my_yaxis={'type':"log",
                  'title':'Population infected'
              }
    traces = []
    #for each in country_list:
    df_plot=df_input_large[df_input_large['country']==country_drop_down]
    df_plot=df_plot[['state','country','confirmed','date']].groupby(['country','date']).agg(np.sum).reset_index()
    df_plot=df_plot['confirmed']
    ydatanew = np.array(df_plot)
    fitted = SIR_model_Large(ydatanew)
    traces = [dict(
            x=np.arange(len(df_plot)),
             y=fitted,
             mode='markers+lines',
             opacity=0.9,
             name='fitted'),        
            dict(x=np.arange(len(df_plot)),
            y=ydatanew,
            mode='markers+lines',
            opacity=0.9,
            name='Original')]
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                xaxis= {'title':'Days',
                       'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }
if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\ASUS\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning:

Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.

